In [0]:
import numpy as np
import random
import torch
from torch.utils.data import DataLoader
from torchvision import transforms

from cleaning import *
from data_loader import *
from extractNet_connected_vgg11 import *
from helper import *
from metrics import *
from trainer import *
from visualizer import *

In [0]:
!rm -rf __MACOSX
!rm -rf *.zip

In [3]:
!wget https://github.com/MNRKhan/aps360-project/raw/master/datasets/data.zip

--2019-03-22 04:32:16--  https://github.com/MNRKhan/aps360-project/raw/master/datasets/data.zip
Resolving github.com (github.com)... 140.82.118.4, 140.82.118.3
Connecting to github.com (github.com)|140.82.118.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/MNRKhan/aps360-project/master/datasets/data.zip [following]
--2019-03-22 04:32:16--  https://raw.githubusercontent.com/MNRKhan/aps360-project/master/datasets/data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9812038 (9.4M) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>]   9.36M  --.-KB/s    in 0.06s   

2019-03-22 04:32:16 (155 MB/s) - ‘data.zip’ saved [9812038/9812038]



In [4]:
!unzip data.zip

Archive:  data.zip
   creating: data/
   creating: data/images/
  inflating: data/images/63.jpg      
  inflating: data/images/189.jpg     
  inflating: data/images/77.jpg      
  inflating: data/images/638.jpg     
  inflating: data/images/604.jpg     
  inflating: data/images/162.jpg     
  inflating: data/images/176.jpg     
  inflating: data/images/88.jpg      
  inflating: data/images/610.jpg     
  inflating: data/images/348.jpg     
  inflating: data/images/360.jpg     
  inflating: data/images/406.jpg     
  inflating: data/images/412.jpg     
  inflating: data/images/374.jpg     
  inflating: data/images/599.jpg     
  inflating: data/images/228.jpg     
  inflating: data/images/214.jpg     
  inflating: data/images/572.jpg     
  inflating: data/images/566.jpg     
  inflating: data/images/200.jpg     
  inflating: data/images/758.jpg     
  inflating: data/images/770.jpg     
  inflating: data/images/764.jpg     
  inflating: data/images/765.jpg     
  inflating: data/images

In [0]:
!rm -rf __MACOSX
!rm -rf *.zip
!rm -rf data_loader.py

In [0]:
# Set random seeds
torch.manual_seed(360)
np.random.seed(360)
random.seed(360)

# Form dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
dataset = ImageMaskDataset("/content/data", transform)

# Dataset sizes
size = len(dataset)

train_size = int(0.6 * size)
valid_size = int(0.2 * size)
test_size = size - train_size - valid_size

# Splitting datasets
train_data, valid_data, test_data = torch.utils.data.random_split(dataset, [train_size, valid_size, test_size])

# Making dataloader
train = DataLoader(train_data, batch_size=2, shuffle=True, num_workers=1)
valid = DataLoader(valid_data, batch_size=2, shuffle=True, num_workers=1)

In [7]:
net = extractNet_connected_vgg11()

Downloading: "https://download.pytorch.org/models/vgg11-bbd30ac9.pth" to /root/.torch/models/vgg11-bbd30ac9.pth
531456000it [00:06, 88450749.54it/s]


In [8]:
trainModel(net, train, valid, batch_size=2, num_epochs=50, checkpoint=True)

Epoch:0, Train IoU:0.4134, Train Loss:0.3486|Valid IoU:0.3680, Valid Loss:0.3652
Epoch:1, Train IoU:0.4019, Train Loss:0.3463|Valid IoU:0.3559, Valid Loss:0.3628
Epoch:2, Train IoU:0.4534, Train Loss:0.3405|Valid IoU:0.3889, Valid Loss:0.3606
Epoch:3, Train IoU:0.5753, Train Loss:0.3253|Valid IoU:0.4734, Valid Loss:0.3561
Epoch:4, Train IoU:0.6000, Train Loss:0.3088|Valid IoU:0.4487, Valid Loss:0.3460
Epoch:5, Train IoU:0.7016, Train Loss:0.2969|Valid IoU:0.5240, Valid Loss:0.3507
Epoch:6, Train IoU:0.7151, Train Loss:0.2818|Valid IoU:0.4694, Valid Loss:0.3343
Epoch:7, Train IoU:0.8045, Train Loss:0.2735|Valid IoU:0.5337, Valid Loss:0.3449
Epoch:8, Train IoU:0.8402, Train Loss:0.2614|Valid IoU:0.4992, Valid Loss:0.3285
Epoch:9, Train IoU:0.8766, Train Loss:0.2601|Valid IoU:0.5289, Valid Loss:0.3382
Epoch:10, Train IoU:0.8992, Train Loss:0.2552|Valid IoU:0.5129, Valid Loss:0.3324
Epoch:11, Train IoU:0.9056, Train Loss:0.2534|Valid IoU:0.4935, Valid Loss:0.3276
Epoch:12, Train IoU:0.9146

KeyboardInterrupt: ignored

In [0]:
# After training (train set)
img, target = train_data[0]
target = target.squeeze(0).detach().numpy()
out = torch.sigmoid(net(img.unsqueeze(0)))
out = out.squeeze(0).squeeze(0).detach().numpy()
imshow(target)
imshow(out)

In [0]:
# After training (validation set)
img, target = valid_data[0]
target = target.squeeze(0).detach().numpy()
out = torch.sigmoid(net(img.unsqueeze(0)))
out = out.squeeze(0).squeeze(0).detach().numpy()
imshow(target)
imshow(out)